# AEDWIP
```
Andrew E. Davidson
aedavids@ucsc.edu
```

ref:
- extraCellularRNA/terra/deseq/doc/plots
- extraCellularRNA/terra/deseq/doc/plots/jupyterNotebooks/GTExValidateExploration.ipynb
- https://support.terra.bio/hc/en-us/articles/360042259232-Managing-data-and-automating-workflows-with-the-FISS-API
- [An introduction to using the Fiss API in Python in BioData Catalyst](https://terra.biodatacatalyst.nhlbi.nih.gov/#workspaces/biodata-catalyst/BioData%20Catalyst%20Collection/notebooks/launch/Intro%20to%20FISS%20API%20in%20Python.ipynb)
- [upsetPlot](https://upsetplot.readthedocs.io/en/stable/api.html#upsetplot.plot)

## initialize

In [1]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%Y-%b-%d %H:%M:%S")
print("Run on ", current_time)

Run on  2022-Jul-31 18:29:38


In [2]:
from firecloud import fiss
import firecloud.api as fapi
import numpy as np
import os
import io
import pandas as pd
from pathlib import Path

# use display() to print an html version of a data frame
# useful if dataFrame output is not generated by last like of cell
from IPython.display import display, Image

In [3]:
tmp = "./tmp"
!mkdir -p $tmp

In [4]:
# set env var used by fiss
# Get the Google billing project name and workspace name
billing_project = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

print("Billing project: " + billing_project)
print("Workspace: " + workspace)
print("Workspace storage bucket: " + bucket)

Billing project: test-aedavids-proj
Workspace: uber
Workspace storage bucket: gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/


In [5]:
# Install a pip package in the current Jupyter kernel
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install --quiet upsetplot

# install upset plot
# see extraCellularRNA/terra/deseq/bin/createUpsetPlotZip.sh
url = bucket + "python/upsetPlot.zip"
! gsutil cp $url .
! unzip -o upsetPlot.zip > /dev/null
ORIG_PYTHONPATH = os.environ['PYTHONPATH']

Copying gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/python/upsetPlot.zip...
/ [1 files][ 13.0 MiB/ 13.0 MiB]                                                
Operation completed over 1 objects/13.0 MiB.                                     


In [6]:
# setting the python path allows us to run python scripts from using
# the CLI. 
PYTHONPATH = ORIG_PYTHONPATH + ":./python"
os.environ["PYTHONPATH"] = PYTHONPATH
PYTHONPATH = os.environ["PYTHONPATH"]
print("PYTHONPATH: {}".format(PYTHONPATH))

# test install. We should get a help message
! python python/plots/geneSignatureUpsetPlot.py

# to be able to import our local python files we need to set the sys.path
# https://stackoverflow.com/a/50155834
sys.path.append(os.getcwd() + '/python')
print("\nsys.path:\n{}\n".format(sys.path))

PYTHONPATH: /etc/jupyter/custom:/usr/lib/spark/python:/home/jupyter/packages:./python
CALLING initializeLogging()
first call initialize Logging
CALLING initializeLogging()
usage: geneSignatureUpsetPlot.py [-h] [-t] [-w WIDTH] [-z HEIG] -d  -o  -c  -i
geneSignatureUpsetPlot.py: error: the following arguments are required: -d/--dataSetsCSV, -o/--outputFile, -c/--numThreads, -i/--intersectionOutputFile

sys.path:
['/home/jupyter/uber/edit', '/etc/jupyter/custom', '/usr/lib/spark/python', '/home/jupyter/packages', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/home/jupyter/.local/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/home/jupyter/.ipython', '/home/jupyter/uber/edit/python']



In [7]:
outDir = "output"
! mkdir -p $outDir

outputImgDir = outDir + "/img"
! mkdir -p $outputImgDir

## configure

In [8]:
def listWorkspaceEntities():
    '''
    prints out the names of all the workspace data models, and the number of rows in each model
    
    returns void
    '''
    # Take a look at all of the data models / entities in of our workspace
    ent_types = fiss.fapi.list_entity_types(billing_project, workspace).json()
    for t in ent_types.keys():
        print (t, "count:", ent_types[t]['count'])
        
listWorkspaceEntities()        

GTEx_TCGA_1vsAll_set count: 7
GTEx_1vsAll_set count: 2
TCGA_1vsAll_set count: 2
GTEx_1vsAll count: 51
GTEx_TCGA_1vsAll count: 83
TCGA_1vsAll count: 32


In [ ]:
class SignatureGeneConfig(object):
    '''
    Class used to ensure parameters for a given data model are in a consistant state
    
    should be treated as constant, immutable values
    '''
    
    def __init__(self, terraDataEntity, design, padjThreshold, lfcThreshold, n, dataOutputBucketRoot,
                localCacheRoot):
        '''
        arguments
            terraDataEntity:
                a name a terra workspace data model returned by listWorkspaceEntities.
                Expected to have a 'candidateSignatureGeneProfile' column
                
            Design:
                a string with the DESeq design. displayed on plots and encoded into data file names
                
            padjThreshold:
                selects genes with padj values <= padjThreshold
                
            lfcThreshold
            
            n: 
                type integer: 
                The number of rows to be select. 
                
            dataOutputBucketRoot
                example: gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/1vsAll/{up|down|best}
                location to store genes of interest
                
            localCacheRoot
                example: output
                
                
                
        '''
        self.terraDataEntity = terraDataEntity
        self.design = design
        self.padjThreshold = padjThreshold
        self.lfcThreshold = lfcThreshold
        self.n = n
        self.dataOutputBucketRoot = dataOutputBucketRoot
        self.localCacheRoot = localCacheRoot
        
        localCache = self.getLocalCachedDir()
        ! mkdir -p $localCache
        
    def getfileNameBase(self):
        tmp = "{}-design:{}-padj:{}-lfc:{}-n:{}".format(
                                self.terraDataEntity,
                                self.design,
                                self.padjThreshold,
                                self.lfcThreshold, 
                                self.n
                                )
        return tmp.replace(" ","_")
        
    def saveGenesOfInterestToBucketURL(self):
        return self.dataOutputBucketRoot + "/" + self.getfileNameBase() 
    
    def getLocalCachedDir(self) :
        return self.localCacheRoot + "/" + self.getfileNameBase()

In [9]:
GTExTCGA_Config_top25 = SignatureGeneConfig(
    terraDataEntity = 'GTEx_TCGA_1vsAll',
    design = "~  gender + category", 
    padjThreshold = 0.001,
    lfcThreshold = 2.0,
    n = 25,
    dataOutputBucketRoot = bucket + "data/1vsAll/up",
    localCacheRoot = outDir
    )

# set signatureGeneConfig = to the data set you want to run
signatureGeneConfig = GTExTCGA_Config_top25

print( signatureGeneConfig.saveGenesOfInterestToBucketURL())
print( signatureGeneConfig.getLocalCachedDir() )

gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/1vsAll/up/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25
output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25


In [10]:
# terraDataEntity = 'GTEx_TCGA_1vsAll'
# GTEx_TCGA_Design = "~  gender + category" 
# designStr = GTEx_TCGA_Design
# PADJ_THRESHOLD = 0.001
#lfcThreshold = 2.0
#topN = 25


#design = "~ sex + tissue_id".replace(" ", "")
# terraDataEntity = "GTEx_1vsAll"
#designStr = "~sex+tissue_id"


# outDir = "output"
# ! mkdir -p $outDir

# outputImgDir = outDir + "/img"
# ! mkdir -p $outputImgDir

# load 1vsAll results

In [11]:
def loadTerraDataModel(billingProject, workspace, modelName) :
    '''
    makes the data models we would see on the terra uber workspace data tab.
    
    returns a pandas dataframe
    '''
    ret = pd.read_csv( io.StringIO(
                                    fiss.fapi.get_entities_tsv(
                                        billing_project, 
                                        workspace, 
                                        modelName,
                                        model='flexible')
                                    .text), 
                              sep='\t')
    return ret

# load the data model that has 1vsAll results
terraDataEntityDF = loadTerraDataModel(billing_project, workspace, signatureGeneConfig.terraDataEntity)
terraDataEntityDF

,entity:GTEx_TCGA_1vsAll_id,candidateSignatureGeneProfile,category,dataSet,estimatedSizeFactors
0,GTEx_Adipose_Subcutaneous,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Adipose_Subcutaneous,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
1,GTEx_Adipose_Visceral_Omentum,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Adipose_Visceral_Omentum,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
2,GTEx_Adrenal_Gland,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Adrenal_Gland,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
3,GTEx_Artery_Aorta,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Artery_Aorta,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
4,GTEx_Artery_Coronary,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Artery_Coronary,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
...,...,...,...,...,...
78,TCGA_THCA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,THCA,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
79,TCGA_THYM,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,THYM,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
80,TCGA_UCEC,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,UCEC,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
81,TCGA_UCS,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,UCS,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...


In [12]:
def select1vsAllResults(df):
    '''
    select rows where candidateSignatureGeneProfile is not null
    
    returns a pandas dataframe
    '''
    selectRowsLogical = ~df.loc[:,'candidateSignatureGeneProfile'].isnull()
    retDF = df.loc[selectRowsLogical, :]
    
    return retDF  

In [13]:
# clean up. remove rows that are missing 1vsAll results
terraDataEntityDF = select1vsAllResults(terraDataEntityDF)
print("{}.shape:{}".format(signatureGeneConfig.terraDataEntity, terraDataEntityDF.shape))
assert terraDataEntityDF.shape[0] == 83, "ERROR: expected 83 candidateSignatureGeneProfiles"
#display( terraDataEntityDF.head() )

print("\n")
for f in terraDataEntityDF.loc[:,'candidateSignatureGeneProfile'].to_list():
    print( f.split("/")[-1] )

terraDataEntityDF

GTEx_TCGA_1vsAll.shape:(83, 5)


Adipose_Subcutaneous_vs_all.results
Adipose_Visceral_Omentum_vs_all.results
Adrenal_Gland_vs_all.results
Artery_Aorta_vs_all.results
Artery_Coronary_vs_all.results
Artery_Tibial_vs_all.results
Bladder_vs_all.results
Brain_Amygdala_vs_all.results
Brain_Anterior_cingulate_cortex_BA24_vs_all.results
Brain_Caudate_basal_ganglia_vs_all.results
Brain_Cerebellar_Hemisphere_vs_all.results
Brain_Cerebellum_vs_all.results
Brain_Cortex_vs_all.results
Brain_Frontal_Cortex_BA9_vs_all.results
Brain_Hippocampus_vs_all.results
Brain_Hypothalamus_vs_all.results
Brain_Nucleus_accumbens_basal_ganglia_vs_all.results
Brain_Putamen_basal_ganglia_vs_all.results
Brain_Spinal_cord_cervical_c-1_vs_all.results
Brain_Substantia_nigra_vs_all.results
Breast_Mammary_Tissue_vs_all.results
Cells_Cultured_fibroblasts_vs_all.results
Cells_EBV-transformed_lymphocytes_vs_all.results
Cervix_Endocervix_vs_all.results
Colon_Sigmoid_vs_all.results
Colon_Transverse_vs_all.results
Esophagus_Gast

,entity:GTEx_TCGA_1vsAll_id,candidateSignatureGeneProfile,category,dataSet,estimatedSizeFactors
0,GTEx_Adipose_Subcutaneous,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Adipose_Subcutaneous,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
1,GTEx_Adipose_Visceral_Omentum,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Adipose_Visceral_Omentum,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
2,GTEx_Adrenal_Gland,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Adrenal_Gland,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
3,GTEx_Artery_Aorta,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Artery_Aorta,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
4,GTEx_Artery_Coronary,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,Artery_Coronary,GTEx,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
...,...,...,...,...,...
78,TCGA_THCA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,THCA,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
79,TCGA_THYM,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,THYM,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
80,TCGA_UCEC,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,UCEC,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...
81,TCGA_UCS,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...,UCS,TCGA,gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/f...


In [14]:
def downLoadCandidateSignatureGeneProfile(cache, dataModelDF):
    '''
    downloads candidateSignatureGeneProfile files to local disk if not already in local cache
    These files are the results created by running 1vsAll.wdl
    
    arguments:
        cache : 
            string path to director to store files location
        
        dataModelDF
            example, GTEx_1vsAllDF
            
    returns
        list of file path on local machine
    '''
    retList = []
    resultsList = dataModelDF.loc[:, 'candidateSignatureGeneProfile'].to_list()
    for gsURL in resultsList:
        fileName = gsURL.split("/")[-1]
        savePath = cache + "/" + fileName
        path = Path(savePath)
        if path.is_file() :
            print("skipping download of {}".format(fileName))            
        else:
            !gsutil -m cp $gsURL $savePath
            
        retList.append(savePath)
            
    return retList
            
        
candidateSignatureFileList = downLoadCandidateSignatureGeneProfile(tmp, terraDataEntityDF)

skipping download of Adipose_Subcutaneous_vs_all.results
skipping download of Adipose_Visceral_Omentum_vs_all.results
skipping download of Adrenal_Gland_vs_all.results
skipping download of Artery_Aorta_vs_all.results
skipping download of Artery_Coronary_vs_all.results
skipping download of Artery_Tibial_vs_all.results
skipping download of Bladder_vs_all.results
skipping download of Brain_Amygdala_vs_all.results
skipping download of Brain_Anterior_cingulate_cortex_BA24_vs_all.results
skipping download of Brain_Caudate_basal_ganglia_vs_all.results
skipping download of Brain_Cerebellar_Hemisphere_vs_all.results
skipping download of Brain_Cerebellum_vs_all.results
skipping download of Brain_Cortex_vs_all.results
skipping download of Brain_Frontal_Cortex_BA9_vs_all.results
skipping download of Brain_Hippocampus_vs_all.results
skipping download of Brain_Hypothalamus_vs_all.results
skipping download of Brain_Nucleus_accumbens_basal_ganglia_vs_all.results
skipping download of Brain_Putamen_basa

# Find Genes of interest

In [15]:
# def findUpRegulatedSignatureGenes(deseqDF, topN=25, lfcThreshold=2.0, padjThreshold=PADJ_THRESHOLD):
#     '''
#     Find genes that that are statistically signifigant and up requlated
    
#     arguments:
#         deseqDF:
#             results of DESeq2 as a pandas dataframe 
            
#         topN:
#             The number of genes to select
#             default = 25
        
#         lfcThreshold:
#             log fold threshold. selectes genes with lfc >=
#             default = 2.0
            
#         padjThreshold:
#             p value
#             default = PADJ_THRESHOLD
    
#     return:
#         pandas dataframe
#     '''
#     selectSignificantRowsPS = deseqDF.loc[:,"padj"] < padjThreshold
# #     print("number of genes with padj < {} : {}".format(padjThreshold,
# #                                                        selectSignificantRowsPS.sum()))

#     deseqLFCSignatureGenesDF = deseqDF.loc[ selectSignificantRowsPS,: ]\
#                                         .sort_values("log2FoldChange", ascending=False)
    
#     # find the genes that are over expresed 
#     selectLFCPS = deseqLFCSignatureGenesDF.loc[:,"log2FoldChange"] >= lfcThreshold
# #     print("number of genes with log2FoldChange >= {} = {}"\
# #             .format( lfcThreshold, selectLFCPS.sum() ))
    
#     deseqBaseMeanSignatureGenesDF = deseqLFCSignatureGenesDF.loc[ selectLFCPS,: ]\
#                             .sort_values("baseMean", ascending=False)
    
#     topSignatureGenesDF = deseqBaseMeanSignatureGenesDF.head( topN )
#     return topSignatureGenesDF

In [16]:
def findUpRegulatedSignatureGenes(deseqDF, signatureGeneConfig):
    '''
    Find genes that that are statistically signifigant and up requlated
    
    arguments:
        deseqDF:
            results of DESeq2 as a pandas dataframe 
            
        signatureGeneConfig
            contains run parmeters            
            
    
    return:
        pandas dataframe
    '''
    selectSignificantRowsPS = deseqDF.loc[:,"padj"] <signatureGeneConfig.padjThreshold
#     print("number of genes with padj < {} : {}".format(padjThreshold,
#                                                        selectSignificantRowsPS.sum()))

    deseqLFCSignatureGenesDF = deseqDF.loc[ selectSignificantRowsPS,: ]\
                                        .sort_values("log2FoldChange", ascending=False)
    
    # find the genes that are over expresed 
    selectLFCPS = deseqLFCSignatureGenesDF.loc[:,"log2FoldChange"] >= signatureGeneConfig.lfcThreshold
#     print("number of genes with log2FoldChange >= {} = {}"\
#             .format( lfcThreshold, selectLFCPS.sum() ))
    
    deseqBaseMeanSignatureGenesDF = deseqLFCSignatureGenesDF.loc[ selectLFCPS,: ]\
                            .sort_values("baseMean", ascending=False)
    
    topSignatureGenesDF = deseqBaseMeanSignatureGenesDF.head( signatureGeneConfig.n )
    return topSignatureGenesDF

In [17]:
# def runFindUpRegulated( csgpFileList, outDir, topN=topN, lfcThreshold=lfcThreshold, 
#                        padjThreshold=PADJ_THRESHOLD, skipRows=7 ):
#     '''
#     finds ups up regulated genes
#     saves to disk as CSV file
    
#     arguments:
#         csgpFileList: 
#             a list of file paths to candidate signature gene files to include in upset plot
            
#         outDir:
#             String
#             path to directory to save up regulated genes
            
#         topN:
#             an integer: 
#             number of genes to select. 
        
#         lfcThreshold:
#             float: 
#             log fold change threshold. example 2.0
            
#         padjThreshold
#             float:
#             p-value threshold. example 0.001
            
#         skipRows:
#             int, default = 7
#             1vsAll returns the results from DESeq with a self describing header comprised of 7 rows
#             the lfcShrink output has 6 rows 
            
#     returns: (upRegulatedDict, outFileList)
#         upRegulatedDict : dictionary
#             key: csgpFile name
#             value: pandas dataframe
        
#     '''
#     retDict = {}
#     retOutFileList = []
#     for csgpFile in csgpFileList:
#         deseqDF = pd.read_csv(csgpFile, skiprows=skipRows)
#         topSignatureGenesDF = findUpRegulatedSignatureGenes(deseqDF,
#                                                             topN=25, 
#                                                             lfcThreshold=2.0,
#                                                             padjThreshold=PADJ_THRESHOLD)

#         fileName = csgpFile.split("/")[-1]
#         outFilePath = outDir + "/" + fileName
#         topSignatureGenesDF.to_csv(outFilePath, index=False)
#         print("saved to file: {}".format(outFilePath))

#         retDict[fileName] = topSignatureGenesDF
#         retOutFileList.append(outFilePath)

    
#     return (retDict, retOutFileList)

In [18]:
def runFindUpRegulated( signatureGeneConfig, candidateSignatureFileList,  skipRows=7 ):
    '''
    finds ups up regulated genes
    
    arguments:
        signatureGeneConfig
            contains run parmeters
            
        candidateSignatureFileList: 
            a list of file paths to candidate signature gene files to include in upset plot
            
            
        skipRows:
            int, default = 7
            1vsAll returns the results from DESeq with a self describing header comprised of 7 rows
            the lfcShrink output has 6 rows             
            
    returns: (upRegulatedDict, outFileList)
        upRegulatedDict : dictionary
            key: csgpFile name
            value: pandas dataframe
        
    '''
    retDict = {}
    retOutFileList = []
    for csgpFile in candidateSignatureFileList:
        deseqDF = pd.read_csv(csgpFile, skiprows=skipRows)
        topSignatureGenesDF = findUpRegulatedSignatureGenes(deseqDF, signatureGeneConfig)
                                                        

        fileName = csgpFile.split("/")[-1]
        outDir = signatureGeneConfig.getLocalCachedDir() 
        outFilePath = outDir + "/" + fileName
        topSignatureGenesDF.to_csv(outFilePath, index=False)
        print("saved to file: {}".format(outFilePath))

        retDict[fileName] = topSignatureGenesDF
        retOutFileList.append(outFilePath)

    
    return (retDict, retOutFileList)

upRegulatedDict, outFileList =  runFindUpRegulated(signatureGeneConfig, candidateSignatureFileList, skipRows=7)

saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adipose_Subcutaneous_vs_all.results
saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adipose_Visceral_Omentum_vs_all.results
saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adrenal_Gland_vs_all.results
saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Artery_Aorta_vs_all.results
saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Artery_Coronary_vs_all.results
saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Artery_Tibial_vs_all.results
saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Bladder_vs_all.results
saved to file: output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Brain_Amygdala_vs_all.results
saved to file: output/GTEx

In [28]:
%%time
# save to long term storage
URL_ROOT = signatureGeneConfig.saveGenesOfInterestToBucketURL() 
print("saving to: {}".format(url))
print()

for f in outFileList:
    url = URL_ROOT + "/" + f
    ! echo gsutil -m cp $f $url

saving to: gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/1vsAll/up/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adrenal_Gland_vs_all.results

gsutil -m cp output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adipose_Subcutaneous_vs_all.results gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/1vsAll/up/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adipose_Subcutaneous_vs_all.results
gsutil -m cp output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adipose_Visceral_Omentum_vs_all.results gs://fc-e15b796f-1abe-4206-ab91-bd58374cc275/data/1vsAll/up/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/output/GTEx_TCGA_1vsAll-design:~__gender_+_category-padj:0.001-lfc:2.0-n:25/Adipose_Visceral_Omentum_vs_all.results
gsutil -m cp o

# Explore the genes of interest

In [20]:
for key in upRegulatedDict.keys():
    df = upRegulatedDict[key]
    print("\n" + key)
    display(df.head(n=2))


Adipose_Subcutaneous_vs_all.results


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
2631,SCD,19314.422968,2.339338,0.101717,22.998473,4.828202e-117,7.658534e-116
888,FASN,16799.560621,2.530867,0.080808,31.319477,2.534326e-215,1.190164e-213



Adipose_Visceral_Omentum_vs_all.results


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
2583,C3,33138.986228,2.237056,0.114028,19.618497,1.074844e-85,1.643047e-84
1837,SCD,19314.422968,2.448700,0.112300,21.804963,2.081741e-105,4.473819e-104



Adrenal_Gland_vs_all.results


...

,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
765,FBLN1,14411.907169,2.397763,0.178757,13.413559,5.036298e-41,2.404865e-39
3240,SLPI,5385.050153,2.402001,0.258488,9.292496,1.507125e-20,1.700898e-19



UCS_vs_all.results


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
5811,DES,89881.666167,2.486514,0.532799,4.666894,3.057872e-06,1.965572e-05
374,ACTC1,14193.843595,5.841343,0.580326,10.065615,7.839607e-24,7.810130e-22



UVM_vs_all.results


,name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
152,EEF2,80410.686591,2.125970,0.095457,22.271588,6.968822e-110,1.678347e-107
1736,VIM,71448.838704,3.094284,0.214690,14.412812,4.298477e-47,9.118612e-46


In [21]:
aediwp move to bucket

SyntaxError: invalid syntax (3367799331.py, line 1)

## create dataSet.csv
This file is an argument to plots/geneSignatureUpsetPlot.py. I defines the set of candidate genes to include
in the upset plot

In [ ]:
def createDataSetCSV(csgpFileList):
    retDF = None
    for csgp in csgpFileList:
        setName = csgp.split("/")[-1]
        filePath = csgp
        numHeaderLines = 8

        tmpDF = pd.DataFrame( {"setName":[setName], 'numHeaderLines':[numHeaderLines], 'filePath':[filePath]} )
        if retDF is None:
            retDF = tmpDF
        else:
            byRows = 0
            retDF = pd.concat([retDF, tmpDF], axis=byRows, ignore_index=True)
            
    return retDF
        
        
dataSetDF = createDataSetCSV(outFileList)
dataSetPath = outDir + "/dataSet.csv"
dataSetDF.to_csv(dataSetPath , index=False)
dataSetDF

## Create Upset Plot

In [ ]:
%%time
title = "{} topN={} Signature Genes, padj < {} lf2c > {} sorted by baseMean".format(terraDataEntity, topN, PADJ_THRESHOLD, lfcThreshold)

print("title: {}".format(title))
name= terraDataEntity + "-" + designStr

baseName = name.replace(" ", "_")
outputImg = outputImgDir + "/" + baseName + ".png"
print("outputImg: {}".format(outputImg))

#IntersectionOutFile =  outDir + "/" + baseName + "Insersections.csv"
IntersectionOutFile =  "{}/{}Insersections.csv".format(outDir, baseName)
print("IntersectionOutFile: {}".format(IntersectionOutFile))
print()


numCPU = os.cpu_count() - 1 # leave a cpu for OS 
# width default 8, height default 3
# width default 10, height default 4 
# 10,4 bad image
# 15,10 better, interseciton is blac
# 20, 15 can almost read the set names inserections are blac
# 25, 20
# 40 40 
# 50 50
# ! python python/plots/geneSignatureUpsetPlot.py \
#     -t "$title" \
#     -d "$dataSetPath" \
#     -o "$outputImg" \
#     --width 75 \
#     --height 75 \
#     --numThreads "$numCPU" \
#     --intersectionOutputFile="$IntersectionOutFile"

In [ ]:
%%time

from plots import geneSignatureUpsetPlot as gs

dataSetsDF = pd.read_csv( dataSetPath )
gsup = gs.GeneSignatureUpsetPlot(dataSetsDF, numThreads=numCPU)

In [ ]:
%%time 
def save():
    print("\n BEGIN PLOT")
    # https://upsetplot.readthedocs.io/en/stable/api.html#upsetplot.plot
    figureWidthInInches = 75
    figureHeightInInches = 75
    fig, subPlotDict = gsup.plot(figureWidthInInches, figureHeightInInches)
    fig.suptitle( title, fontsize=8 )  # arial is not installed on courtyard, default font is huge
    fig.savefig(outputImg, dpi=300, bbox_inches='tight')
    print("saved plot: {}".format(outputImg))
    print("END PLOT\n")
 
save()

In [ ]:
%%time
import upsetplot as upsp

plotData = gsup.getUpSetPlotData()
# 2       398
# 3       621
# 4      1058
# 5      1615
# 6      1894
# 7.     1632
# 8 sets 1017
# 9       448
# 10      133
# 11       24
# 12        2
# number of intersection created by 8 or more sets 1017 + 448 + 133 + 24 + 2
upsp.plot( plotData.plotData, show_counts=True, min_degree=8)

intersectionDF = gsup.getIntersectionDF()
intersectionDF.to_csv(IntersectionOutFile, index=False)
print("\n*************** wrote file: {}".format(IntersectionOutFile))

In [ ]:
%%time
geneSetsDict = gsup.getGeneSets()
from upsetplot import from_contents
geneSetsUpsetPlotData = from_contents(geneSetsDict)

from upsetplot import UpSet
#plt = UpSet(geneSetsUpsetPlotData, subset_size='count', min_degree=8).plot()
plt = UpSet(geneSetsUpsetPlotData, show_counts=True, min_degree=8).plot()

In [ ]:
%%time
#plt = UpSet(geneSetsUpsetPlotData, subset_size='count', max_degree=7).plot()
plt = UpSet(geneSetsUpsetPlotData, show_counts=True, max_degree=7).plot()

In [ ]:
plt = UpSet(geneSetsUpsetPlotData, show_counts=True, max_degree=2).plot()

In [ ]:
plt = UpSet(geneSetsUpsetPlotData, show_counts=True, min_degree=3, max_degree=5).plot()

In [ ]:
plt = UpSet(geneSetsUpsetPlotData, show_counts=True, min_degree=6, max_degree=7).plot()

In [ ]:
%%time
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(10,3))
UpSet.plot(geneSetsUpsetPlotData, fig, element_size=None, subset_size='count', max_degree=7)

## explore the intersection

In [ ]:
plotData = gsup.getUpSetPlotData()
plotData.plotData

In [ ]:
plotData.intersectionDict

In [ ]:
print(len(plotData.intersectionDict))

In [ ]:
aedwip

In [ ]:
! ls -l $IntersectionOutFile
! ls -l $outputImg

In [ ]:
display( Image(filename=outputImg) )

## Save

In [ ]:
target = bucket + "upsetPlots/"
print(target)
! gsutil -m cp $outputImg $IntersectionOutFile $target

## Explore the intersections

In [ ]:
adwip wrong file

! ls -l geneSignatureUpsetPlot.Intersection.csv
intersectionDF = pd.read_csv("geneSignatureUpsetPlot.Intersection.csv")
intersectionDF.loc[:, ['tissueId', 'gene', 'log2FoldChange', 'baseMean', 'padj']]